In [2]:
import pymongo as pym
import json
import pandas as pd
import sys
sys.path.insert(0, './lib_python/')

In [43]:
def mongoimport(csv_path, db_name, coll_name, db_url='localhost', db_port=27017, remove = False, sep =","):
    """ Imports a csv file at path csv_name to a mongo colection
    returns: count of the documants in the new collection
    """
    client = pym.MongoClient(db_url, db_port)
    db = client[db_name]
    coll = db[coll_name]
    data = pd.read_csv(csv_path, sep=sep)
    data.rename(columns=lambda x: x.strip())
    print("readed file.. converting to json")
    payload = json.loads(data.to_json(orient='records'))
    print("done")
    
    print("adding point origin and destination...")
    for i, p in enumerate(payload):
        payload[i]["origin_point"] = {"type":"Point", "coordinates":[p["origin_lon"],p["origin_lat "]]}
        payload[i]["dest_point"] = {"type":"Point", "coordinates":[p["dest_lon "],p["dest_lat "]]}
    print("done")
    coll.create_index([("origin_point",pym.GEOSPHERE)])
    coll.create_index([("dest_point",pym.GEOSPHERE)])
    coll.create_index([("origin_point",pym.GEOSPHERE), ("dest_point",pym.GEOSPHERE)])
    if remove:
        coll.delete_many({})
        print("drop collections")
    print("inserting {0} elements in mongo db...".format(len(payload)))
    coll.insert_many(payload)
    print("done inserted {0} in {1} collection with {2} existing elements".format(
        len(payload), coll_name, coll.count() - len(payload)))
    return coll.count()

In [44]:
csv_path = './data/data/car2go/car2go_Milano_01.csv'
db_name = "bigDive_test"
coll_name = "bookings"
db_url = 'localhost'
db_port = 27017
mongoimport(csv_path, db_name, coll_name, db_url, db_port, remove=True, sep=";")

readed file.. converting to json
done
adding point origin and destination...
done


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: ensure_index is deprecated. Use create_index instead.


drop collections
inserting 157578 elements in mongo db...
done inserted 157578 in bookings collection with 0 existing elements


157578

In [47]:
client = pym.MongoClient(db_url, db_port)
db = client[db_name]
coll = db[coll_name]
coll.find_one()

{'_id': ObjectId('5b33afcf9e35ba12fdb81195'),
 'car_name': '-',
 'city': 'Milano',
 'dest_lat ': 45.47628,
 'dest_lon ': 9.16837,
 'dest_point': {'coordinates': [9.16837, 45.47628], 'type': 'Point'},
 'driving_distance': 2019,
 'driving_duration': 397,
 'final_address': 'Via Antonio Canova, 14, 20145 Milano MI',
 'final_date': '2018-01-01 00:15:40',
 'final_fuel': 100,
 'final_time': 1514762140,
 'id': 415950,
 'init_address': 'Via Bullona, 16, 20154 Milano MI',
 'init_date': '2018-01-01 00:04:12',
 'init_fuel': 100,
 'init_time': 1514761452,
 'origin_lat ': 45.48653,
 'origin_lon': 9.16155,
 'origin_point': {'coordinates': [9.16155, 45.48653], 'type': 'Point'},
 'plate': '531/FL427CG',
 'pt_arrival ': 1514763324,
 'pt_distance ': 1894,
 'pt_duration ': 974,
 'vendor': 'car2go',
 'vin': 'WME4530421Y146718',
 'walking_distance': 1470,
 'walking_duration ': 1100}